In [1]:
from collections import deque

In [2]:
cups_input = "284573961"

In [3]:
test_cups_input = "389125467"

In [4]:
def parse_cups(s):
    return [int(d) for d in s]

In [5]:
test_cups = parse_cups(test_cups_input)

In [6]:
cups = parse_cups(cups_input)

In [7]:
def play(cups, rounds):
    cups = deque(cups)
    
    for i in range(rounds):  
        # see what current cup is
        current = cups[0]
        
        # target label based on current cup
        target = current - 1
        
        # rotate once so next three cups are at front
        cups.rotate(-1)
        
        # grab them
        a, b, c = cups.popleft(), cups.popleft(), cups.popleft()
        
        # look for the index of our target. subtract 1 as necessary if we have the cup separate
        while target == a or target == b or target == c or target == 0:
            target = (target - 1) % 10
        
        target_loc = cups.index(target)
        
        # rotate until one after our target
        cups.rotate(-(target_loc+1))
        
        # put the cups we removed back on
        cups.extendleft([c, b, a])

        # rotate back until the cup right after the current cup is first
        current_index = cups.index(current)
        
        cups.rotate(-(current_index+1))
        
        # ready for next iteration (next current cup at front)
    
    # get number string after 1
    one_index = cups.index(1)
    cups.rotate(-one_index)
    cups.popleft()

    s = ''.join([str(d) for d in cups])

    return s

In [8]:
assert play(test_cups, 10) == '92658374'

In [9]:
assert play(test_cups, 100) == '67384529'

In [10]:
assert play(cups, 100) == '26354798'

Can't reuse the solution in pt1. Too slow. Just going to make a circular linked list

In [94]:
class Cup:
    def __init__(self, label):
        self.label = label
        self.next = None

    def __repr__(self):
        return f"Cup label: {self.label}"

In [100]:
def pt2(cups):
    NUM_CUPS = 1_000_000
    NUM_ROUNDS = 10_000_000

    cup_dict = {i: Cup(i) for i in cups} 
    cup_dict.update({i: Cup(i) for i in range(10, NUM_CUPS + 1)})
    
    # now connect them up
    for i, l in enumerate(cups[:-1]):
        cup_dict[l].next = cup_dict[cups[i+1]]
    
    for l in range(10, NUM_CUPS):
        cup_dict[l].next = cup_dict[l+1]
        
    # last normal cup to first of sequential cups
    cup_dict[cups[-1]].next = cup_dict[10]

    # last of sequential cups to first of normal cups
    cup_dict[NUM_CUPS].next = cup_dict[cups[0]]
        
    current_cup = cup_dict[cups[0]]
    
    for move in range(1, NUM_ROUNDS+1):
        # target label based on current cup
        target_label = current_cup.label - 1

        # save the four cups after the current_cup
        # the fourth will be the next current_cup
        a = current_cup.next
        b = a.next
        c = b.next
        d = c.next

        # look for the label of our target.
        # subtract 1 as necessary if we have the cup separated already
        while target_label == a.label or target_label == b.label or target_label == c.label or target_label == 0:
            target_label = (target_label - 1) % (NUM_CUPS + 1)

        target_cup = cup_dict[target_label]

        c.next = target_cup.next

        target_cup.next = a

        current_cup.next = d

        current_cup = d
        
    one_cup = cup_dict[1]
    
    return one_cup.next.label * one_cup.next.next.label

In [101]:
pt2(test_cups)

149245887792

In [102]:
pt2(cups)

166298218695